In [1]:
from aidds.sys.utils.exception import AiddsException
from aidds.modeling.scaler_test import Scaling 
from aidds.modeling.learn_test import Learning
import aidds.sys.config as cfg
from aidds.sys.utils.evaluation import regression_evals
import pandas as pd

In [2]:
try:
    sc = Scaling(is_best=True)
    ln = Learning(scaling_data=sc.sdata)
except AiddsException as ae:
    ae.print()

[615031e31ce0][2024-04-16 21:14:36.647685] Preprocessing data scaling start.
[615031e31ce0][2024-04-16 21:14:36.785296]   Total size of attribute data for learning: (10968, 99)
[615031e31ce0][2024-04-16 21:14:36.795124]   Data size of training/testing data by pole count: pc_type[all] - total(10968, 99), train(8226, 99), test(2742, 99)
[615031e31ce0][2024-04-16 21:14:36.845774]   Data size of training/testing data by pole count: pc_type[e1] - total(6611, 99), train(4958, 99), test(1653, 99)
[615031e31ce0][2024-04-16 21:14:36.886669]   Data size of training/testing data by pole count: pc_type[n1] - total(4357, 99), train(3267, 99), test(1090, 99)
[615031e31ce0][2024-04-16 21:14:41.498591] Preprocessing data scaling end., Total processing time: 0:00:04.850927
[7c1f51102e7f][2024-04-16 21:14:41.498712] Learning start.
[7c1f51102e7f][2024-04-16 21:14:41.634867]   Model training results: Data[ALL] - MAPE: 0.499806, R2SCORE: 0.570910
[7c1f51102e7f][2024-04-16 21:14:41.641450]   Model training

In [3]:
ln._best

{'all': RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=1234),
 'e1': ElasticNet(alpha=0.1, random_state=1234),
 'n1': RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=1234)}

In [4]:
model = ln._best['all']['model']
ddict = {did: ln._sdict[did]['all'] for did in cfg.type.mds.ids}

TypeError: list indices must be integers or slices, not str

In [ ]:
train_x = pd.concat([ddict['train_x'], ddict['test_x']])
train_y = pd.concat([ddict['train_y'], ddict['test_y']])

In [ ]:
train_x.shape

(10968, 99)

In [ ]:
modeling_cols = ddict['train_x'].columns.tolist()[1:]

In [ ]:
pred_y = model.predict(train_x[modeling_cols])

In [ ]:
train_y = train_y.to_numpy()

In [ ]:
evals, message = \
    regression_evals(y=train_y, p=pred_y, verbose=1)

In [ ]:
message

'MAPE: 0.505013, R2SCORE: 0.585906'